In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
raw=pd.read_csv('train.csv')

In [3]:
raw.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [4]:
raw[['OpenPorchSF','EnclosedPorch','3SsnPorch','ScreenPorch']].head()

,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch
0,61,0,0,0
1,0,0,0,0
2,42,0,0,0
3,35,272,0,0
4,84,0,0,0


In [5]:
raw['pri_per']=raw['SalePrice']/raw['LotArea']

In [6]:
def handle_mv(col):
    raw[col]=np.where(raw[col].isnull()==True,'None',raw[col])
columns=['Alley','BsmtQual','BsmtCond','BsmtExposure','BsmtFinType1','BsmtFinType2','FireplaceQu','GarageType','GarageFinish','GarageQual','GarageCond','PoolQC','Fence','MasVnrType']
handle_mv(columns)

In [7]:
raw['age']=raw['YrSold']-raw['YearBuilt']
raw['re_age']=raw['YrSold']-raw['YearRemodAdd']
raw['re_built']=raw['YearRemodAdd']-raw['YearBuilt']

In [8]:
raw['porch']=np.nan
for porch in [col for col in raw.columns if 'Porch' in col]:
    raw['porch'].fillna(raw[porch],inplace=True)


In [9]:
valuecount={}
for idex, col in enumerate(raw.columns):
    valuecount[col]= raw[col].value_counts()
    

In [11]:
condition=raw[['Condition1','Condition2']]
condition=pd.get_dummies(condition )
condition.insert(loc=16, column='Condition2_RRNe', value=0)
list1=list(condition.columns[:9])
list2=list(condition.columns[-9:])
list3=list(condition.columns)

In [12]:
level=['Artery','Feedr','Norm','PosA','PosN','RRAe','RRAn','RRNe','RRNn']
list_condition=list(zip(list1,list2,level))
for i in range(len(list_condition)):
    condition[list_condition[i][2]]=condition[list_condition[i][0]]+condition[list_condition[i][1]]
    

In [13]:
def two_one(col):
    condition[col]=[1 if x==2 else 1 if x==1 else 0 for x in condition[col]]
for i in range(len(level)):
    two_one(level[i])

In [14]:
condition=condition.drop(columns=list3)

In [15]:
raw=pd.concat([raw, condition], axis=1)

In [16]:
raw['Functional']=[1 if x=='Typ' else 0  for x in raw['Functional']]
raw['PavedDrive']=[1 if x=='Y' else 0  for x in raw['PavedDrive']]
raw['PoolQC']=[0 if x=='None'else 1 for x in raw['PoolQC']]
raw['Fence'] = [0 if x==None else 1 if x=='MnPrv'or x=='MnWw'else 2 for 
                     x in raw['Fence']]

In [17]:
raw=raw.drop(columns=['Condition1','Condition2','Utilities','OverallQual','Exterior2nd','MasVnrArea','ExterCond','BsmtFinType1',
                     'BsmtFinSF1','BsmtFinType2','BsmtFinSF2','BsmtUnfSF','Heating','Electrical','BsmtFullBath','BsmtHalfBath',
                     'BedroomAbvGr','KitchenAbvGr','Fireplaces','GarageFinish','GarageArea','GarageQual',
                     'GarageCond','YrSold','SaleType'])
raw=raw.drop(columns=['MSSubClass','PoolArea','1stFlrSF','2ndFlrSF','MiscFeature','GarageYrBlt'])
raw=raw.drop(columns=['OpenPorchSF','EnclosedPorch','3SsnPorch','ScreenPorch'])


In [18]:
raw.shape

(1460, 60)

In [19]:
import statsmodels.api as sm 

new=raw.dropna(subset=['LotFrontage', 'LotArea'])
X = new["LotArea"] 
y = new["LotFrontage"]
X = sm.add_constant(X) 


model = sm.OLS(y, X).fit() 
predictions = model.predict(X)


model.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            LotFrontage   R-squared:                       0.182
Model:                            OLS   Adj. R-squared:                  0.181
Method:                 Least Squares   F-statistic:                     266.0
Date:                Fri, 22 Feb 2019   Prob (F-statistic):           3.70e-54
Time:                        21:37:27   Log-Likelihood:                -5414.3
No. Observations:                1201   AIC:                         1.083e+04
Df Residuals:                    1199   BIC:                         1.084e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         57.0550      1.018     56.024      0.000      55.057      59.053
LotArea        0.0013   8.01e-05     16.309      0.000       0.001       0.001
==============================================================================
Omnibus:                      456.762   Durbin-Watson:                   2.030
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            14409.128
Skew:                           1.114   Prob(JB):                         0.00
Kurtosis:                      19.822   Cond. No.                     2.04e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.04e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [20]:
 raw['LotFrontage']=np.where(raw['LotFrontage'].isnull()==True,raw['LotArea']*0.0013+57.055,raw['LotFrontage'])

In [21]:
raw.isnull().any(axis=0)

Id               False
MSZoning         False
LotFrontage      False
LotArea          False
Street           False
Alley            False
LotShape         False
LandContour      False
LotConfig        False
LandSlope        False
Neighborhood     False
BldgType         False
HouseStyle       False
OverallCond      False
YearBuilt        False
YearRemodAdd     False
RoofStyle        False
RoofMatl         False
Exterior1st      False
MasVnrType       False
ExterQual        False
Foundation       False
BsmtQual         False
BsmtCond         False
BsmtExposure     False
TotalBsmtSF      False
HeatingQC        False
CentralAir       False
LowQualFinSF     False
GrLivArea        False
FullBath         False
HalfBath         False
KitchenQual      False
TotRmsAbvGrd     False
Functional       False
FireplaceQu      False
GarageType       False
GarageCars       False
PavedDrive       False
WoodDeckSF       False
PoolQC           False
Fence            False
MiscVal          False
MoSold     

In [22]:
raw['porch'].isnull().sum()

0

In [23]:
raw['porch'].head()

0    61.0
1     0.0
2    42.0
3    35.0
4    84.0
Name: porch, dtype: float64